# Linear Regression Project: Predicting House Sale Prices

We will be working with Housing data for the city of Ames, Iowa, United States from 2006 to 2010.<br>
Links: https://doi.org/10.1080/10691898.2011.11889627<br>
Data Description: https://s3.amazonaws.com/dq-content/307/data_description.txt

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [3]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,41.0,4920,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,39.0,5389,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,60.0,7500,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,189000


In [4]:
df[['Gr Liv Area','SalePrice']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 2 columns):
Gr Liv Area    2930 non-null int64
SalePrice      2930 non-null int64
dtypes: int64(2)
memory usage: 45.9 KB


### Create a set of functions to automate the Linear Regression

In [5]:
def transform_features(df):
    return df
    
def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test[features], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

### Test the functions

In [6]:
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

print(rmse)

190668.1183522945


## Feature Engineering

- Handle missing values:
    - All columns:
        - Drop any with 5% or more missing values <b> for now</b>
    - Text columns:
        - Drop any with 1 or more missing values <b> for now</b>
    - Numerical columns:
        - For columns with missing values, fill in with the most common value in that column
        
1: All columns: Drop any with 5% or more missing values<b> for now</b>

In [7]:
num_missing = df.isnull().sum()
num_missing

Order                0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       490
Lot Area             0
Street               0
Alley             2732
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        23
Mas Vnr Area        23
Exter Qual           0
Exter Cond           0
                  ... 
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual

In [8]:
drop_missing_cols = num_missing[(num_missing > len(df) * 0.05)].sort_values()

df = df.drop(drop_missing_cols.index, axis=1)

In [9]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,4920,Pave,Reg,Lvl,AllPub,Inside,...,170,0,0,0,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,...,0,0,144,0,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,5389,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,7500,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,189000


Dropped 12 columns

2: Text columns: Drop any with 1 or more missing values <b> for now</b> 

In [10]:
# number of text missing values columns
text_mv_counts = df.select_dtypes(include='object').isnull().sum().sort_values(ascending=False)

drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

df = df.drop(drop_missing_cols_2.index, axis=1)

In [11]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,4920,Pave,Reg,Lvl,AllPub,Inside,...,170,0,0,0,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,...,0,0,144,0,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,5389,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,7500,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,6,2010,WD,Normal,189000


Dropped 7 columns

3: Numerical columns: For columns with missing values, fill in with the most common value in that column - mode

In [12]:
# Compute column-wise missing value counts
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(df) * 0.05) & (num_missing > 0)].sort_values()

fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [13]:
replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]

replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [14]:
df = df.fillna(replacement_values_dict)

In [15]:
df.isnull().sum().value_counts()

0    64
dtype: int64

All columns have no more missing values

In [16]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [17]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod <0]

1702   -1
2180   -2
2181   -1
dtype: int64

Create new columns with these features<br>
Drop rows with negative values for both of these features<br>
Remove original year columns

In [18]:
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

df = df.drop([1702,2180,2181],axis=0)

df = df.drop(['Year Built', 'Year Remod/Add'], axis=1)

Drop columns that:
- aren't useful for ML
- leak data about the final sale - <a href="https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt"/>here

In [19]:
# Useless columns
df = df.drop(['PID','Order'], axis=1)

# Columns that leak info about the final sale
#Mo Sold: Month Sold (MM)
#Yr Sold: Year Sold (YYYY)
#Sale Condition: Condition of sale
#Sale Type: Type of sale
df = df.drop(['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)

Update transform_features()

In [20]:
def transform_features(df):
    # Numerical columns
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df) * 0.05)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    # Text columns
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    # Fixable numerical columns
    num_missing = df.select_dtypes(include=['int','float']).isnull().sum().sort_values()
    fixable_numeric_cols = num_missing[(num_missing < len(df) * 0.05) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    # New Modified Years Features
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181], axis=0)
    
    df = df.drop(['PID','Order','Mo Sold','Sale Condition','Sale Type','Year Built','Year Remod/Add'], axis=1)
    return df
    

In [21]:
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

189574.4585807409

## Feature Selection

In [22]:
numerical_df = transform_df.select_dtypes(include=['int','float'])
numerical_df.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,62,0,0,0,0,0,2010,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,0,0,0,120,0,0,2010,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,36,0,0,0,0,12500,2010,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,2010,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,34,0,0,0,0,0,2010,189900,13,12


In [23]:
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
Yr Sold              0.030358
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice 

As an experiment, we will only keep columns with a correlation coefficient of larger than 0.4

In [24]:
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

Drop columns with less than 0.4 correlation with SalePrice

In [25]:
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs > 0.4].index, axis=1)

Identify columns to keep, and columns that are meant to be categorical.

In [26]:
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

- Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
- If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [41]:
# Which categorical columns have we still carried with us?
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

# How many unique values in each categorical columns?

# def count_val(col):
#     return len(col.value_counts())
# uniqueness_counts = transform_df[transform_cat_cols].apply(count_val).sort_values()

uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col : len(col.value_counts())).sort_values()
# ARBITRUARY CUTOFF OF 10 UNIQUE VALUES (Worth Experimenting)
drop_non_unique_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_non_unique_cols, axis=1)

In [42]:
transform_df

,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Condition 1,Condition 2,...,Functional,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold
0,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,Typ,P,210,62,0,0,0,0,0,2010
1,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,...,Typ,Y,140,0,0,0,120,0,0,2010
2,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,Typ,Y,393,36,0,0,0,0,12500,2010
3,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,Typ,Y,0,0,0,0,0,0,0,2010
4,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,212,34,0,0,0,0,0,2010
5,RL,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,360,36,0,0,0,0,0,2010
6,RL,4920,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,0,0,170,0,0,0,0,2010
7,RL,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,0,82,0,0,144,0,0,2010
8,RL,5389,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,237,152,0,0,0,0,0,2010
9,RL,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,Typ,Y,140,60,0,0,0,0,0,2010


Dropped a few more columns that were categorical, and had 10 unique categories or less
- Preventing a low variance numerical feature

In [46]:
# Select the remaining text columns and convert them to categorical

text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
# Create dummy columns and add back to dataframe
transform_df = pd.concat([
    transform_df,
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols, axis=1)

In [64]:
transform_df.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Area', 'Street', 'Lot Shape',
       'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood',
       'Condition 1', 'Condition 2', 'Bldg Type', 'House Style',
       'Overall Qual', 'Overall Cond', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'BsmtFin SF 1', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area',
       'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd',
       'Functional', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
       'Screen Porch', 'Pool Area', 'Misc Val', 'Yr Sold', 'SalePrice',
       'Years Before Sale', 'Years Since Remod'],
      dtype='objec

### Update select_features()
#### Put all functions here

In [68]:
def transform_features(df):
    # Numerical columns
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df) * 0.05)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    # Text columns
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    # Fixable numerical columns
    num_missing = df.select_dtypes(include=['int','float']).isnull().sum().sort_values()
    fixable_numeric_cols = num_missing[(num_missing < len(df) * 0.05) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    # New Modified Years Features
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702,2180,2181], axis=0)
    
    df = df.drop(['PID','Order','Mo Sold','Sale Condition','Sale Type','Year Built','Year Remod/Add'], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, unique_threshold=10):
    numerical_df = df.select_dtypes(include=['int','float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col : len(col.value_counts())).sort_values()
    drop_non_unique_cols = uniqueness_counts[uniqueness_counts > unique_threshold].index
    df = df.drop(drop_non_unique_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    df = pd.concat([
        df,
        pd.get_dummies(df.select_dtypes('category'))
    ], axis=1).drop(text_cols,axis=1)
    
    return df

In [69]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['int', 'float'])
    features = numeric_df.columns.drop('SalePrice')
    lr = LinearRegression()
    
    if k == 0:
        train = numeric_df[:1460]
        test = numeric_df[1460:]

        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test[features], predictions)
        rmse = np.sqrt(mse)
        
        return rmse
        
    elif k == 1:
        # Randomize all rows from df
        shuffled_df = df.sample(frac=1)
        fold_one = shuffled_df[:1460]
        fold_two = shuffled_df[1460:]
        
        lr.fit(fold_one[features], fold_one['SalePrice'])
        predictions_one = lr.predict(fold_two[features])
        mse_one = mean_squared_error(fold_two[features], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        predictions_twp = lr.predict(fold_one[features])
        mse_two = mean_squared_error(fold_one[features])
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    
    else: # k more than 1
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
        

In [70]:
df = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[29664.22358435856, 32861.34350506342, 30600.40209588086, 39839.19003569408]


33241.28980524923

### Potential Next Steps:
- Continue iteration on feature engineering
- Research on othr approaches to feature engineering online around <b>housing data</b>
- Visit <a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels"> Kaggle</a> for this dataset to see approaches others took
- Improve feature selection
    - Research ways of doing feature selection better with categorical columns

See the <a href="https://community.dataquest.io/tags/c/social/share/49/240"> Community's Work </a>